In [30]:
import json
import http.client
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

True

In [26]:
def get_token(access_key): 
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ''
    headers = {
    'AccessKey': access_key
    }
    conn.request("GET", "/uraDataService/insertNewToken.action", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']

In [55]:
def get_result(token, access_key, route):
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ""
    headers = {
        "AccessKey": access_key,
        "Token": token,
        "User-Agent": "PostmanRuntime/7.26.8",
    }
    conn.request("GET", route, payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']


def get_all_results(token, access_key, routes):
    result = []
    for route in routes:
        result.extend(get_result(token, access_key, route))
    return pd.DataFrame(result)


In [57]:
private_transactions_routes = [
    "/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=" + str(i)
    for i in range(1, 5)
]
private_rental_routes = [
    "/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod=14q1"
]  # start from 14q1 to 23q1
planning_decisions_routes = [
    "/uraDataService/invokeUraDS?service=Planning_Decision&year=" + str(i)
    for i in range(2000, 2024)
]

access_key = os.environ.get("URA_ACCESS_KEY")
token = get_token(access_key)

get_all_results(token, access_key, private_transactions_routes).to_csv(
    "../data/private_transactions.csv", index=False
)
get_all_results(token, access_key, private_rental_routes).to_csv(
    "../data/private_rental.csv", index=False
)
get_all_results(token, access_key, planning_decisions_routes).to_csv(
    "../data/planning_decisions.csv", index=False
)
